In [1]:
import os
import urllib.request
import zipfile
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import time
from datetime import datetime

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es = Elasticsearch('elasticsearch-node:9200')

if es.indices.exists('stream-test'):
    es.indices.delete('stream-test')
    
    body={
        'mappings': {
            'properties': {
                'count': {'type': 'text'},
                'name': {'type': 'text'},
                'value': {'type': 'text'},
                'timestamp': {'type': 'text'}
            }
        }
    }
    
    es.indices.create(index='stream-test', body=body)
    
print(es.info())

{'name': '927d8d22d8c7', 'cluster_name': 'docker-cluster', 'cluster_uuid': '038uq0CNScCWYvn2HwLEoA', 'version': {'number': '7.5.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '8bec50e1e0ad29dad5653712cf3bb580cd1afcdf', 'build_date': '2020-01-15T12:11:52.313576Z', 'build_snapshot': False, 'lucene_version': '8.3.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [4]:
# set environment variable PYSPARK_SUBMIT_ARGS
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-hadoop-7.5.2/dist/elasticsearch-spark-20_2.11-7.5.2.jar pyspark-shell'

# invoke SparkContext (using environment variable)
sc = SparkContext(appName="PythonSparkStreaming")

In [5]:
es_write_conf = {

    # specify the node that we are sending data to (this should be the master)
    "es.nodes" : '927d8d22d8c7',

    # specify the port in case it is not the default port
    "es.port" : '9200',

    # specify a resource in the form 'index/doc-type'
    "es.resource" : 'stream-test',

    # is the input JSON?
    "es.input.json" : "yes",

    # is there a field in the mapping that should be used to specify the ES document ID
    "es.mapping.id": "_doc"

}

In [10]:
print(os.getcwd())

with open('./sample/stream-sample0.9774294006781428.json', 'r') as data:
    print(data)
    

/home/jovyan
<_io.TextIOWrapper name='./sample/stream-sample0.9774294006781428.json' mode='r' encoding='UTF-8'>
